In [1]:
!pip install datasets -q -U
!pip install pyarrow==15.0.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 45.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 16.1.0
    Uninstalling pyarrow-16.1.0:
      Successfully uninstalled pyarrow-16.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but 

In [8]:
!pip install trl -q
!pip install peft -q
!pip install --upgrade transformers -q
!pip install --upgrade bitsandbytes -q

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import torch
import json

from transformers import pipeline
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from transformers import BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer, DPOTrainer, DPOConfig

ModuleNotFoundError: No module named 'peft'

In [5]:
from huggingface_hub import login
HF_TOKEN =  "YOUR TOKEN"
login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
splits = {'train': 'data/train-00000-of-00001-b42a775f407cee45.parquet', 'validation': 'data/validation-00000-of-00001-134b8fd0c89408b6.parquet'}
df_train = pd.read_parquet("hf://datasets/OpenAssistant/oasst1/" + splits["train"])
df_val = pd.read_parquet("hf://datasets/OpenAssistant/oasst1/" + splits["validation"])

instructs_train = df_train[(df_train.lang == 'ru') & (df_train['parent_id'].isna())]
instructs_val = df_val[(df_val.lang == 'ru') & (df_val['parent_id'].isna())]
instructs_train.head()

,message_id,parent_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,rank,synthetic,model_name,detoxify,message_tree_id,tree_state,emojis,labels
82,e480f611-0d31-433a-93d2-0e2bc675aa30,None,05a999f8-8219-4584-a3f2-d4b4aec30027,2023-02-06T23:09:05.356836+00:00,Напиши информацию о игре Hytale,prompter,ru,3,True,False,NaN,False,None,"{'toxicity': 0.0008757906034588814, 'severe_to...",e480f611-0d31-433a-93d2-0e2bc675aa30,ready_for_export,"{'name': ['+1', '_skip_ranking', '_skip_labeli...","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
185,fa8c92d6-6daa-42a3-a2f5-e2e0eb610c41,None,19241e3c-8272-4c3f-afe8-527c005d042c,2023-02-07T14:36:48.863674+00:00,"Докончи поговорку, без труда не выловишь и",prompter,ru,3,True,False,NaN,False,None,"{'toxicity': 0.0432540699839592, 'severe_toxic...",fa8c92d6-6daa-42a3-a2f5-e2e0eb610c41,ready_for_export,"{'name': ['+1', '-1'], 'count': [3, 1]}","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
315,ef69d986-aa2d-4c3e-a122-1910d9625c79,None,6b1efbfa-7c16-47bf-b90c-340ab2b1b7eb,2023-02-08T14:59:45.846381+00:00,"У меня угнали машину, что мне делать?",prompter,ru,3,True,False,NaN,False,None,"{'toxicity': 0.004925671499222517, 'severe_tox...",ef69d986-aa2d-4c3e-a122-1910d9625c79,ready_for_export,"{'name': ['+1', '_skip_reply'], 'count': [4, 1]}","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
327,e321f8f6-01f8-4975-afe6-73db989f538e,None,3c8cde8c-9466-494f-9c36-cbf3c3ac7ecd,2023-02-08T14:19:18.286994+00:00,Какие произведения Булгакова связаны с Иерусал...,prompter,ru,3,True,False,NaN,False,None,"{'toxicity': 0.0015869040507823229, 'severe_to...",e321f8f6-01f8-4975-afe6-73db989f538e,ready_for_export,"{'name': ['+1', '_skip_reply', '_skip_ranking'...","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
427,5083658c-e251-4259-b180-77afaf907d66,None,05a999f8-8219-4584-a3f2-d4b4aec30027,2023-02-07T15:32:50.130297+00:00,Плюсы и минусы языков программирования C++ и R...,prompter,ru,3,True,False,NaN,False,None,"{'toxicity': 0.0003262242244090885, 'severe_to...",5083658c-e251-4259-b180-77afaf907d66,ready_for_export,"{'name': ['+1', '_skip_reply'], 'count': [6, 3]}","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."


# Baseline

Сгенерируем ответы предобученной модели. В качестве базовой модели возьмем модель saiga_llama3_8b, т.к.

1. Она одна из немногих хорошо генерирует на русском языке
2. Имеет оптимальный размер
3. Хорошо управляется промптами
4. Также лучше показала себя в сравнении с gemma-2-2b-it и Llama-3.2-3B-Instruct

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
)
tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga_llama3_8b")
model = AutoModelForCausalLM.from_pretrained("IlyaGusev/saiga_llama3_8b", 
                                             device_map="cuda:0", 
                                             quantization_config=bnb_config)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

In [12]:
instructs_val_tokenized = []
for instruct in list(instructs_val.text):
    instructs_val_tokenized.append(tokenizer(instruct, return_tensors="pt", truncation=True)['input_ids'].to('cuda'))
    
generation_output = []
for instruct in tqdm(instructs_val_tokenized, desc="Generating answers"):
    generation_output.append(model.generate(input_ids=instruct))

Generating answers: 100%|██████████| 39/39 [1:30:32<00:00, 139.28s/it]


In [13]:
base_answers_dict = {"prompt": [], "generated_text": []}
for i, answer in enumerate(generation_output):
    answer = tokenizer.decode(answer[0])
    prompt = instructs_val.text.iloc[i]
    generated_text = answer[answer.find(prompt) + len(prompt):]
    base_answers_dict['prompt'].append(prompt)
    base_answers_dict['generated_text'].append(generated_text)
base_answers_dict

{'prompt': ['Напиши функцию на языке swift, которая сортирует массив целых чисел, а затем выводит его на экран',
  'Чему равен абсолютный ноль?',
  'Что такое сверхпроводимость?',
  'Напиши пошаговый план как сделать ракету в гараже?',
  'Смотри, есть такой постироничный мем - глубокомысленные цитаты про волков.\n\n"Лучше быть последним — первым, чем первым — последним." "Если волк молчит, то лучше его не перебивать." "Волк слабее льва и тигра, но в цирке не выступает" "Волк не волк, когда волк волк"\n\nПридумай пять цитат в таком же стиле',
  'Привет, скажи своё мнение по поводу войны в Украине',
  'Опиши OpenAssistant как SCP-объект',
  "Слушай, Ася, мне нужен твой совет. Меня уже порядочно достал Сбербанк своими спамовыми SMS'ками. Особенно бесит, когда предлагают даже не кредиты и кредитки, а какой-то трэш, связанный с их барахолкой, которой я не пользовался и не имею ни малейшего желания. Есть ли какой-нибудь способ добиться того, чтобы от этого банка мне приходили только сообщени

В целом, видно, что ответы достаточно подробные, однако мы попытаемся дальше с помощью alignment сделать их ещё более подробными, охватывающими все темы в запросе и по возможности содержащими ссылки на надёжный источник. 

# DPO
Используя метку rank в датасете построим тройки (промпт, лучший ответ, худший ответ) и подадим на вход стандартному алгоритму alignment -- DPO

In [6]:
def get_top_level_answers(df_train, instructs_train):
    top_level_answers= df_train[df_train.parent_id.isin(list(instructs_train.message_id))].groupby("parent_id")[['rank', 'text']].aggregate(lambda x: list(x)).reset_index()
    top_level_answers['highest_rank'] = top_level_answers['rank'].apply(lambda x: np.argmin(x))
    top_level_answers['lowest_rank'] = top_level_answers['rank'].apply(lambda x: np.argmax(x))
    top_level_answers['prompt'] = top_level_answers.parent_id.apply(lambda x: instructs_train[instructs_train.message_id == x].text.values[0])
    return top_level_answers

top_level_train = get_top_level_answers(df_train, instructs_train)
top_level_val = get_top_level_answers(df_val, instructs_val)
top_level_train.head()

,parent_id,rank,text,highest_rank,lowest_rank,prompt
0,00672f0b-f1cd-4a86-b8b6-a159fe39b222,"[0.0, 1.0]",[Конечная реализация парсинга XML будет зависе...,0,1,напиши парсинг xml
1,00d6d94e-bb0b-4dd3-99b2-e059227582a5,"[0.0, 1.0, 2.0]",[Килограмм камней и килограмм пуха весят одина...,0,2,Что тяжелее килограмм камней или килограмм пуха?
2,022de87e-4ec9-4fe3-a673-449c4fad7ab2,"[0.0, 1.0, 2.0]",[В большинстве случаев горение воды или чего-л...,0,2,как сделать огонь внутри воды
3,030c7749-7b6d-497a-8eb9-5eea31737358,"[0.0, 1.0, 2.0]",[Конечно! Для написания конкретной функции мне...,0,2,Создай функцию на пайтон
4,0349af51-4b07-4a7e-bde8-9e868d42957a,"[0.0, 1.0, 2.0]","[Конечно, я могу помочь с рецептом яичницы! Во...",0,2,"Я хотел бы пожарить яичницу. \nНапиши, пожалуй..."


In [7]:
def get_dpo_dataset(top_level_answers):
    best_answers = pd.DataFrame(top_level_answers[['text', 'highest_rank']].apply(lambda row: row[0][row[1]], axis=1)).rename({0: 'answer'}, axis=1)
    best_answers['prompt'] = top_level_answers['prompt']
    worst_answers = pd.DataFrame(top_level_answers[['text', 'lowest_rank']].apply(lambda row: row[0][row[1]], axis=1)).rename({0: 'answer'}, axis=1)
    worst_answers['prompt'] = top_level_answers['prompt']
    return best_answers.merge(worst_answers, on="prompt").rename({"answer_x": "chosen", "answer_y": "rejected"}, axis=1)

train_dataset = get_dpo_dataset(top_level_train)
val_dataset = get_dpo_dataset(top_level_val)
train_dataset.head()

/tmp/ipykernel_30/2351924707.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  best_answers = pd.DataFrame(top_level_answers[['text', 'highest_rank']].apply(lambda row: row[0][row[1]], axis=1)).rename({0: 'answer'}, axis=1)
/tmp/ipykernel_30/2351924707.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  worst_answers = pd.DataFrame(top_level_answers[['text', 'lowest_rank']].apply(lambda row: row[0][row[1]], axis=1)).rename({0: 'answer'}, axis=1)
/tmp/ipykernel_30/2351924707.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as l

,chosen,prompt,rejected
0,Конечная реализация парсинга XML будет зависет...,напиши парсинг xml,"К сожалению, я не могу написать полный код для..."
1,Килограмм камней и килограмм пуха весят одинак...,Что тяжелее килограмм камней или килограмм пуха?,Вес килограмма камней эквивалентен весу килогр...
2,В большинстве случаев горение воды или чего-ли...,как сделать огонь внутри воды,Как вариант - можно зажечь огонь в отделенном ...
3,Конечно! Для написания конкретной функции мне ...,Создай функцию на пайтон,Какую ?
4,"Конечно, я могу помочь с рецептом яичницы! Вот...","Я хотел бы пожарить яичницу. \nНапиши, пожалуй...","Возможно, я вас удивлю, но куриное яйцо соверш..."


In [8]:
training_args = DPOConfig(
    output_dir="./output",
    beta=0.1,
    max_length=1512,
    max_prompt_length=1024,
    remove_unused_columns=False,
)

peft_config = LoraConfig(
        lora_alpha=128,
        r=8,
)

In [11]:
dpo_trainer = DPOTrainer(
    model,
    None,
    args=training_args,
    train_dataset=Dataset.from_pandas(train_dataset),
    eval_dataset=Dataset.from_pandas(val_dataset),  
    tokenizer=tokenizer,
    peft_config=peft_config
)

dpo_trainer.train()

Tokenizing train dataset:   0%|          | 0/723 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/39 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


OutOfMemoryError: CUDA out of memory. Tried to allocate 840.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 221.12 MiB is free. Process 4156 has 15.67 GiB memory in use. Of the allocated memory 15.19 GiB is allocated by PyTorch, and 199.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

К сожалению, 16GB видеокарты на kaggle не хватило для обучения несмотря на то, что к ней была применена квантизация
и что работа была не в сетапе full-training, ведь почти все параметры модели были предварительно заморожены


# Подход из Self-Rewarding Language Models
Он предполагает собой итеративное дообучение:

0. Модель М1 учится на IFT+EFT данных с помощью SFT на основе базовой модели М0
1. М1 генерирует промпты и ответы на них и сама их ранжирует, собирая тем самым синтетические пары ответов для DPO; обучается и из неё получается M2
2. M2 генерирует данные для обучения DPO точно так же, как в п.1, на которых обучается М3.
3. и т.д.

IFT данные -- это данные для Instruction Fine Tuning, в нашем случае -- наш исходный датасет в формате (промпт (=инструкция), лучший ответ на неё по мнению асессоров).

EFT данные -- это данные для Evaluation Fine Tuning, который формируется следующим образом: берём пары (промпт, ответ 1 на него), (промпт, ответ 2 на него) и т.д., просим модель их оценить по заранее данным ей критериям. Если её ранжирование совпадает с асессорским -- добавляем эти пары в тренировочную выборку.





## Подготовка данных

In [9]:
ift_data = best_answers.copy()

In [10]:
system_prompt = f'''
Ты помогаешь размечать данные. 
Оцени приведённый ответ ассистента <ANSWER> на запрос пользователя <USER> по пятибалльной шкале:
- В ответе должна содержаться полная, исчерпывающая информация по запросу пользователя.
- Ответ должен содержать явную или неявную ссылку на надёжный источник.
- Информация в ответе должна быть написана простым языком, лёгким для восприятия.
Верни только одно число -- свою оценку в формате: "ОЦЕНКА: <ТВОЯ ОЦЕНКА>" Также приведи короткое обоснование своего мнения (длиной не больше 50 слов). Не разбирай сам ответ <ANSWER>.
'''

def get_user_prompt(answer, query):
    return f"<USER>: <{query}>\n<ANSWER>: <{answer}>"

evaluation_prompts = []
for i, row in top_level_answers.iterrows():
    prompt, text = row['prompt'], row['text']
    for answer in text:
        evaluation_prompts.append(system_prompt + get_user_prompt(answer, prompt))
len(evaluation_prompts)

1948

In [11]:
evaluation_prompts_tokenized = []
for instruct in list(evaluation_prompts):
    evaluation_prompts_tokenized.append(tokenizer(instruct, return_tensors="pt", truncation=True)['input_ids'].to('cuda'))
    
evaluation_scores = []
for instruct in tqdm(evaluation_prompts_tokenized[:1], desc="Generating scores"):
    evaluation_scores.append(model.generate(input_ids=instruct))

Generating scores: 100%|██████████| 1/1 [02:25<00:00, 145.49s/it]


In [15]:
# Находим оценки, которые модель поставила генерациям
scores = []
for i, answer in enumerate(evaluation_scores):
    prompt = evaluation_prompts[i]
    answer = tokenizer.decode(answer[0])
    try:
        score = int(re.findall(r'ОЦЕНКА: \d', answer)[0][-1])
        response_start = re.search(r'ОЦЕНКА: \d', answer).start()
        response = answer[response_start:]
    except IndexError:
        score = int(re.findall(r'Оценка: \d', answer)[0][-1])
        response_start = re.search(r'Оценка: \d', answer).start()
        response = answer[response_start:]
    scores.append((score, prompt, response))

In [16]:
# Преобразуем их в более удобный для дальнейшего обучения вид
scores_pd = []
j = 0
for i, row in top_level_answers.iterrows():
    prompt, text = row['prompt'], row['text']
    prompt_scores = []
    for answer in text:
        try:
            prompt_scores.append(scores[j])
        except IndexError:
            break
        j += 1
    scores_pd.append(prompt_scores)

In [17]:
# Формируем eft данные: если ранжирование модели на промпте и относящимся к нему ответам совпало с асессорским,
# то кладём пару (промпт, ответ) в eft.
eft_data = {"prompt": [], "answer": []}
for prompt_scores in scores_pd:
    if len(prompt_scores) == 0:
        break
    prompt_max_score, prompt_min_score = max(prompt_scores, key=lambda x: int(x[0])), min(prompt_scores, key=lambda x: x[0])[0]
    if prompt_scores[0][0] == prompt_max_score[0]:
        eft_data['prompt'].append(prompt_scores[0][1])
        eft_data['answer'].append(prompt_scores[0][2])
    if prompt_scores[-1][0] == prompt_max_score[0]:
        eft_data['prompt'].append(prompt_scores[-1][1])
        eft_data['answer'].append(prompt_scores[-1][2])

In [18]:
ift_eft_data = Dataset.from_pandas(pd.concat([ift_data, pd.DataFrame(eft_data)], axis=0))

## SFT

In [19]:
lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    r=4,
    lora_alpha=32,
)

In [20]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,703,936 || all params: 8,031,965,184 || trainable%: 0.0212


In [21]:
training_args = SFTConfig(
    dataset_text_field="answer",
    max_seq_length=512,
    output_dir="/tmp"
)

trainer = SFTTrainer(
    model,
    train_dataset=ift_eft_data,
    args=training_args,
)

trainer.train()


Map:   0%|          | 0/723 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 21.12 MiB is free. Process 3016 has 15.87 GiB memory in use. Of the allocated memory 14.14 GiB is allocated by PyTorch, and 1.43 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

К сожалению, несмотря на все мои попытки ускорить обучение модели и уменьшить саму модель, её обучение всё равно не влезло в 16GB GPU на kaggle. Других альтернатив этой модели особо нет, т.к. в задании сказано, что модель должна хорошо уметь генерировать ответы на русском, что сразу сильно сужает класс вариантов. К тому же, учитывая то, что она должна хорошо воспринимать промпты (не игнорируя информацию оттуда, как это делает та же gemma-2-2b-it) и действительно менять свои ответы в соотстветствии с изменениями в промптах, этот класс ещё сильнее сужается до 1-2 моделей. Однако тем не менее я изучала статьи и различные подходы, но у меня, к сожалению, не хватило ресурсов как минимум сравнить все эти методы, а как максимум реализовать другие мои идеи.